In [ ]:
from audioop import cross

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import numpy.lib.recfunctions as rf

In [ ]:
hsi_fm = pd.read_csv('hsi_fm.csv', index_col = 0, parse_dates = True)
hhi_fm = pd.read_csv('hhi_fm.csv', index_col = 0, parse_dates = True)

In [ ]:
hsi_fm.head()

In [ ]:
hsi_array = hsi_fm.drop(columns = 'SPREAD').reset_index().rename(columns={'index': 'TIMESTAMP'}).to_records(index=False)

# General Functions

In [ ]:
def lagArray(data, lag=1):
    """
    Lag a numpy array by a specified number of steps.
    
    Args:
        data (numpy.ndarray): The input array.
        lag (int): The lag step (default is 1).
    
    Returns:
        numpy.ndarray: The lagged array.
    """
    # Convert to float to handle NaN values
    data = data.astype(float)
    
    lagged_data = np.empty_like(data)
    lagged_data[:lag] = np.nan  # Fill the beginning of the array with NaNs
    lagged_data[lag:] = data[:-lag]
    
    # Replace NaN with 0
    lagged_data = np.nan_to_num(lagged_data, nan=0.0)

    return lagged_data

def mapSignaltoPosition(signal: np.array, asset: np.array, cash_delta_lim: int) -> np.array:
    """
    Transforms a continuous signal into the desired position in the asset.
    
    Parameters:
        - signal (numpy.ndarray): The lagged input signal.
        - asset (numpy.ndarray): The input asset.
        - cash_delta_lim (int): The cash delta limit.
    
    Returns:
        - np.array: The transformed asset array with the desired position.
    """
    
    # adds the signal to the existing array
    asset = rf.append_fields(asset, 'SIGNALS', signal, usemask=False)
    
    # calculate the desired cash delta position at each time
    asset = rf.append_fields(asset, 'CASH_DELTA_TARGET', asset['SIGNALS'] * cash_delta_lim, usemask=False)
    
    # calculate the number of stocks required to satisfy targeted cash delta, rounding towards zero
    asset = rf.append_fields(asset, 'UNITS', np.where(asset['CASH_DELTA_TARGET'] >= 0, np.floor(asset['CASH_DELTA_TARGET'] / asset['LAST_PRICE']), np.ceil(asset['CASH_DELTA_TARGET'] / asset['LAST_PRICE'])).astype(int), usemask=False)
    
    # calculating the actual cash delta position
    asset = rf.append_fields(asset, 'CASH_DELTA_ACTUAL', asset['UNITS'] * asset['LAST_PRICE'], usemask=False)
    
    # calculating the difference in units, or the trade that needs to be done
    asset = rf.append_fields(asset, 'UNITS_DIFF', np.insert(np.diff(asset['UNITS']), 0, 0), usemask=False)
    
    return asset

def tradeCD(asset: np.ndarray) -> np.ndarray:
    """
    Generates trades required to meet cash delta target and adds the POSITION, TRADED, ACTION, and PRICE columns to the input asset array.
    
    Parameters:
        asset (np.ndarray): The input asset with the signals, target cash delta, and last available bid/ask prices and volume.
    
    Returns:
        np.ndarray: The updated asset array with the POSITION, TRADED, ACTION, and PRICE columns.
    """
    # Initialize the POSITION array with the first entry as 0
    position = np.zeros(len(asset), dtype=int)
    
    # Initialize the TRADED array
    traded = np.zeros(len(asset), dtype=int)
    
    # Initialize the ACTION and PRICE arrays
    action = np.empty(len(asset), dtype='<U4')  # 'BUY', 'SELL', or 'None'
    price = np.empty(len(asset), dtype=float)  # Prices or NaN for 'None'
    
    for i in range(1, len(asset)):
        units = asset[i]['UNITS']
        last_ask_volume = asset[i]['LAST_ASK_VOLUME_0']
        last_bid_volume = asset[i]['LAST_BID_VOLUME_0']
        
        if units > position[i-1]:
            traded[i] = min(units - position[i-1], last_ask_volume)
            action[i] = 'BUY'
            price[i] = asset[i]['LAST_ASK_PRICE_0']
        elif units < position[i-1]:
            traded[i] = -min(position[i-1] - units, last_bid_volume)
            action[i] = 'SELL'
            price[i] = asset[i]['LAST_BID_PRICE_0']
        else:
            traded[i] = 0
            action[i] = 'None'
            price[i] = np.nan
        
        # Update the POSITION for the next iteration
        position[i] = position[i-1] + traded[i]
    
    # Create the updated structured array with new columns
    updated_dtype = asset.dtype.descr + [('POSITION', 'i4'), ('TRADED', 'i4'), ('ACTION', 'U4'), ('PRICE', 'f4')]
    updated_asset = np.zeros(len(asset), dtype=updated_dtype)
    
    # Copy the original data
    for name in asset.dtype.names:
        updated_asset[name] = asset[name]
    
    # Add the new columns
    updated_asset['POSITION'] = position
    updated_asset['TRADED'] = traded
    updated_asset['ACTION'] = action
    updated_asset['PRICE'] = price
    
    return updated_asset

def povertyBacktester(lagged_signal: np.array, asset: np.array) -> np.array:
    """
    Quick backtesting for proof-of-concept signals.
    
    Parameters:
        - lagged_signal (numpy.ndarray): The lagged input signal.
        - asset (numpy.ndarray): The input asset.
    
    Returns:
        - np.array: Series of returns
    """
    
    log_rets = np.diff(np.log(asset['LAST_PRICE']))
    log_rets = np.insert(log_rets, 0,0)
    
    returns = np.cumsum(log_rets * lagged_signal)
    
    return returns

def povertyAnalytics(returns: np.array, asset: np.array, lagged_signal: np.array) -> np.array:
    """
    Quick portfolio analytics. Make sure to change number of minutes in a year etc. 
    
    Parameters:
        - returns (np.array): Series of returns.
        - assets (np.array): Input assets.
        - lagged_signal (np.array): Lagged input signal.
    
    Returns:
        - Summary statistics of portfolio
    """
    
    sharpe = (np.mean(returns) * 60 * 18 * 256) / (np.std(returns) * np.sqrt(60 * 18 * 256))
    sharpe = (np.mean(returns)) / (np.std(returns))
    
    turnover = np.sum(np.abs(np.diff(lagged_signal)))
    
    return sharpe, turnover

def toTrades(asset: np.ndarray) -> pd.DataFrame:
    """
    Filters the input numpy array for rows where 'TRADED' is not equal to 0,
    and extracts the 'TIMESTAMP', 'PRICE', 'TRADED', and 'ACTION' columns,
    converting the result to a pandas DataFrame.
    
    Parameters:
        asset (np.ndarray): The input asset array with the required columns.
    
    Returns:
        pd.DataFrame: DataFrame containing the filtered rows and specified columns.
    """
    # Filter the array for rows where 'TRADED' is not equal to 0
    filtered_asset = asset[asset['TRADED'] != 0]
    
    # Extract the specified columns
    extracted_columns = filtered_asset[['TIMESTAMP', 'PRICE', 'TRADED', 'ACTION']]
    
    # Convert the extracted data to a pandas DataFrame
    df = pd.DataFrame(extracted_columns)
    
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP']).dt.tz_localize('Asia/Hong_Kong')
    
    return df

# Moving Averages and Crossovers

## Above/Below SMA

In [ ]:
def aboveMovingAverage(prices: np.ndarray, period: int) -> np.ndarray:
    """
    Signal for whether asset is above its moving average. 
    
    Parameters:
        - prices (numpy.ndarray): A numpy array consisting of price data for one asset
        - period (int): Defined lookback period for the moving average
    
    Returns:
        - numpy.ndarray: An array consisting of the signals for the asset
    """
    # Calculate the rolling moving average
    rolling_avg = np.convolve(prices, np.ones(period)/period, mode='valid')
    
    # Pad the beginning of rolling_avg to match the length of data
    rolling_avg = np.concatenate((np.full(period-1, np.nan), rolling_avg))
    
    # Generate the output array
    result = np.where(prices > rolling_avg, 1, -0)
    
    return rolling_avg, result

In [ ]:
ma_example = aboveMovingAverage(hsi_array['LAST_PRICE'], 20)
ma_example

In [ ]:
mapSignaltoPosition(lagArray(ma_example[1]), hsi_array, 1e6)

In [ ]:
# we can see that the best results appear to be with the 1000, 2000, 4000 periods
# we will flag these for further research with the official backtester

# 1000
_, signal1000 = aboveMovingAverage(hsi_array['LAST_PRICE'], 1000)
signal1000 = lagArray(signal1000)
array1000 = mapSignaltoPosition(signal1000, hsi_array, 1e6)
array1000 = tradeCD(array1000)

# 2000
_, signal2000 = aboveMovingAverage(hsi_array['LAST_PRICE'], 2000)
signal2000 = lagArray(signal2000)
array2000 = mapSignaltoPosition(signal2000, hsi_array, 1e6)
array2000 = tradeCD(array2000)

# 4000
_, signal4000 = aboveMovingAverage(hsi_array['LAST_PRICE'], 4000)
signal4000 = lagArray(signal4000)
array4000 = mapSignaltoPosition(signal4000, hsi_array, 1e6)
array4000 = tradeCD(array4000)

## Moving Average Crossover

In [ ]:
def crossoverMovingAverage(prices: np.ndarray, period1: int, period2: int) -> np.ndarray:
    """
    Signal for whether the shorter-period moving average is above the longer-period moving average.
    
    Parameters:
        - prices (numpy.ndarray): A numpy array consisting of price data for one asset.
        - period1 (int): The first lookback period for the moving average (shorter period).
        - period2 (int): The second lookback period for the moving average (longer period).
    
    Returns:
        - tuple: Tuple containing the two moving averages and the signal array.
    """
    # Calculate the moving averages for both periods
    rolling_avg1 = np.convolve(prices, np.ones(period1)/period1, mode='valid')
    rolling_avg2 = np.convolve(prices, np.ones(period2)/period2, mode='valid')
    
    # Pad the beginning of the moving averages to match the length of data
    rolling_avg1 = np.concatenate((np.full(period1-1, np.nan), rolling_avg1))
    rolling_avg2 = np.concatenate((np.full(period2-1, np.nan), rolling_avg2))
    
    # Generate the signal array
    signal = np.where(np.isnan(rolling_avg1) | np.isnan(rolling_avg2), 0, np.where(rolling_avg1 > rolling_avg2, 1, -1))
    
    return rolling_avg1, rolling_avg2, signal

# Example
crossoverMovingAverage(hsi_array['LAST_PRICE'], 20, 50)[2][:100]

## ensTrend

In [ ]:
def ema(prices, span):
    """
    Calculate the Exponential Moving Average (EMA).
    
    Parameters:
        prices (np.ndarray): Array of price data.
        span (int): Span for the EMA.
    
    Returns:
        np.ndarray: Array of EMA values.
    """
    alpha = 2 / (span + 1)
    ema = np.zeros_like(prices)
    ema[0] = prices[0]
    for i in range(1, len(prices)):
        ema[i] = alpha * prices[i] + (1 - alpha) * ema[i - 1]
    return ema

def ensTrend(prices: np.ndarray) -> np.ndarray:
    """
    Takes a series of prices, processes trend following signals based on exponentially-weighted moving average crossovers, and outputs position signals.
    
    Parameters:
        prices (np.ndarray): Array of back-adjusted price data for different assets.
        
    Returns:
        np.ndarray: Trend following signals scaled from -20 to 20.
    """
    
    # Calculate EMAs
    ema2 = np.apply_along_axis(ema, 0, prices, 2 * 24 * 60)
    ema4 = np.apply_along_axis(ema, 0, prices, 4 * 24 * 60)
    ema8 = np.apply_along_axis(ema, 0, prices, 8 * 24 * 60)
    ema16 = np.apply_along_axis(ema, 0, prices, 16 * 24 * 60)
    ema32 = np.apply_along_axis(ema, 0, prices, 32 * 24 * 60)
    
    # Calculate moving average differences
    emac2 = (ema2 - ema8) * 5.95 / prices
    emac4 = (ema4 - ema16) * 4.10 / prices
    emac8 = (ema8 - ema32) * 2.79 / prices
    
    # Create raw forecasts
    raw_forecast = 25 * (emac2 + emac4 + emac8) / 3
    
    # Cap the forecasts
    capped_forecast = np.clip(raw_forecast, -20, 20)
    capped_forecast[0] = 0  # Set the first value to 0
    
    return capped_forecast

# Example 
ensTrend(hsi_array['LAST_PRICE'])

# Breakouts

## Individual

In [ ]:
def breakoutIndividual(prices: np.ndarray, period: int) -> np.ndarray:
    """
    Generates signals for the Breakout strategy for a specific period
    
    Parameters:
        - prices (numpy.ndarray): A numpy array consisting of price data for one asset
        - period (int): Defined lookback period for the price range for the breakout signal
    
    Returns:
        - numpy.ndarray: An array consisting of the signals for the asset
    """
    
    # Convert the numpy array to a pandas Series
    prices_series = pd.Series(prices)
    
    # Calculate the rolling mean, max and min for the series
    rolling_mean = prices_series.rolling(period, min_periods=1).mean()
    rolling_max = prices_series.rolling(period, min_periods=1).max()
    rolling_min = prices_series.rolling(period, min_periods=1).min()
        
    # Calculate the raw forecast
    raw_forecast = 2 * (prices_series - rolling_mean) / (rolling_max - rolling_min)
    
    # Smooth the forecast
    smoothed_forecast = raw_forecast.ewm(span=period / 4, adjust=False).mean()
    
    # Convert the smoothed forecast to a numpy array
    result = smoothed_forecast.to_numpy()
    
    return result

# Example
breakoutIndividual(hsi_array['LAST_PRICE'], period=1000)

In [ ]:
def prodBreakoutIndiv(asset: np.ndarray, period: int, cdl: int = 1e6) -> np.ndarray:
    """
    Full pipeline from signals to trades. 
    
    Parameters:
        - asset (numpy.ndarray): A numpy array consisting of price data for one asset
        - period (int): Defined lookback period for the price range for the breakout signal
        - cdl (int): Cash delta limit
    
    Returns:
        - numpy.ndarray: An array consisting of the signals for the asset
    """
    
    price = asset['LAST_PRICE']
    
    signal = breakoutIndividual(price, period)
    signal = lagArray(signal)
    
    map = mapSignaltoPosition(signal, asset, cdl)
    map = tradeCD(map)
    
    return map

prodBreakoutIndiv(hsi_array, 2000)

## Ensemble

In [ ]:
def breakoutEnsemble(prices: pd.DataFrame, periods: list) -> pd.DataFrame:
    """
    Generates signals for the Breakout strategy for multiple periods.
    
    Parameters:
        prices (pd.DataFrame): A DataFrame consisting of x observations of different assets.
        periods (list): A list of periods for the moving averages.
    
    Returns:
        pd.DataFrame: A DataFrame consisting of the signals for each of the assets.
    """
    
    # Initialize an array to store the sum of forecasts
    average_forecast = np.zeros_like(prices, dtype=float)
    
    # Apply breakoutIndividual to each period and add the results to average_forecast
    for period in periods:
        forecast = np.apply_along_axis(breakoutIndividual, 0, prices, period)
        average_forecast += forecast
    
    # Divide by the number of periods to get the average
    average_forecast /= len(periods)
    
    # Convert the numpy array back to a DataFrame
    
    return average_forecast

# Example
breakoutEnsemble(hsi_array['LAST_PRICE'], [1000, 2000, 4000])

In [ ]:
def prodBreakoutEnsemble(asset: np.ndarray, periods: list, cdl: int = 1e6) -> np.ndarray:
    """
    Full pipeline from signals to trades. 
    
    Parameters:
        - asset (numpy.ndarray): A numpy array consisting of price data for one asset
        - period (list): Defined lookback period for the price range for the breakout signal
        - cdl (int): Cash delta limit
    
    Returns:
        - numpy.ndarray: An array consisting of the signals for the asset
    """
    
    price = asset['LAST_PRICE']
    
    signal = breakoutEnsemble(price, periods)
    signal = lagArray(signal)
    
    map = mapSignaltoPosition(signal, asset, cdl)
    map = tradeCD(map)
    
    return map

# Example
prodBreakoutEnsemble(hsi_array, [1000, 2000, 4000])

# Signal Processing
see `mapSignaltoPosition`

# Trading Towards Cash Delta

In [ ]:
sample_signals = breakoutIndividual(hsi_array['LAST_PRICE'], 1000)
sample_signals = lagArray(sample_signals)

sample_map = mapSignaltoPosition(sample_signals, hsi_array, 1e6)

In [ ]:
sample_trades = tradeCD(sample_map)
sample_trades

In [ ]:
trades_df = toTrades(sample_trades)
trades_df

In [ ]:
# Plot the first 200 periods
plt.figure(figsize=(10, 6))
plt.plot(sample_trades['UNITS'][:200], label = 'Desired Position')
plt.plot(sample_trades['POSITION'][:200], label = 'Actual Position')
plt.plot(sample_trades['TRADED'][:200], label = 'Units Traded')

plt.xlabel('Row Index')
plt.ylabel('Units')
plt.title('Units Held over First 200 Periods')
plt.legend()
plt.show()


# Step Functions / Portfolio Optimisation

In [ ]:
tan_scalar = np.tanh(1)
adj_weights = np.tanh(sample_signals) / tan_scalar
adj_weights